# DriveNet Open-Loop Evaluation

### what is open-loop evaluation?
When we perform open-loop evaluation of our DriveNet model we evaluate its prediction as we follow the annotated ground truth.

In each frame, we compare the prediction of our model against the ground truth. This can be done with different metrics, such as ADE (Average Displacement Error) or FDE (Final Displacement Error). 

**Regardless of the metric used, thie evaluation protocol doesn't modify the future locations according to our predictions**

TODO add a picture to explain it maybe?


**Note: to generate a model to use here please refer to the training notebook**


In [ ]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, draw_trajectory
from l5kit.drivenet.model import DriveNetModel
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

import os

## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./drivenet_config.yaml")
print(cfg)

## Load The Model



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load("/var/folders/7f/mb3llzfs40b86yf2km8fml2h0000gp/T/drivenet.pt").to(device)
model = model.eval()

## Load the Evaluation Data

In [ ]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

In [ ]:
# ==== EVAL LOOP
position_preds = []
yaw_preds = []

position_gts = []
yaw_gts = []

torch.set_grad_enabled(False)
idx = 0

for data in tqdm(eval_dataloader):
    del data["host_id"]
    result = model(data)
    position_preds.append(result["positions"].detach().cpu().numpy())
    yaw_preds.append(result["positions"].detach().cpu().numpy())

    position_gts.append(data["target_positions"].detach().cpu().numpy())
    yaw_gts.append(data["target_yaws"].detach().cpu().numpy())
    idx += 1
    if idx == 10:
        break
    
position_preds = np.concatenate(position_preds)
position_gts = np.concatenate(position_gts)

### Quantitative Evaluation: ADE and FDE

TODO avail
TODO yaw

In [ ]:
pos_errors = np.linalg.norm(position_preds - position_gts, axis=-1)

# DISPLACMENT AT T
plt.plot(np.arange(pos_errors.shape[1]), pos_errors.mean(0), label="Displacement error at T")
plt.legend()
plt.show()

# ADE HIST
plt.hist(pos_errors.mean(-1), bins=100, label="ADE Histogram")
plt.legend()
plt.show()

# FDE HIST
plt.hist(pos_errors[:,-1], bins=100, label="FDE Histogram")
plt.legend()
plt.show()

# Qualitative Evaluation: Visual Plots


### Visualise Results
We can also visualise some results from the ego (AV) point of view for those frames of interest (the 100th of each scene).

However, as we chopped off the future from the dataset **we must use the GT csv if we want to plot the future trajectories of the agents**


In [ ]:
for frame_number in range(0, len(eval_dataset), len(eval_dataset) // 20):
    
    data = eval_dataloader.dataset[frame_number]
    del data["host_id"]

    data_batch = default_collate([data])
    
    result = model(data_batch)
    predicted_positions = result["positions"].detach().cpu().numpy().squeeze()

    im_ego = rasterizer.to_rgb(data["image"].transpose(1, 2, 0))
    target_positions = data["target_positions"]
    
    predicted_positions = transform_points(predicted_positions, data["raster_from_agent"])
    target_positions = transform_points(target_positions, data["raster_from_agent"])
    
    draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)

    plt.imshow(im_ego[::-1])
    plt.show()